# NGS Data formats and QC

## Introduction
There are several file formats for storing Next Generation Sequencing (NGS) data. In this tutorial we will look at some of the most common formats for storing NGS reads and variant data. We will cover the following formats:

__FASTQ__ - This format stores unaligned read sequences with base qualities  
__SAM/BAM__ - This format stores unaligned or aligned reads (text and binary formats)  
__CRAM__ - This format is similar to BAM but has better compression than BAM  
__VCF/BCF__ - Flexible variant call format for storing SNPs, indels, structural variations (text and binary formats)   

Further to understanding the different file formats, it is important to remember that all sequencing platforms have technical limitations that can introduce biases in your sequencing data. Because of this it is very important to check the quality of the data before starting any analysis, whether you are planning to use something you have sequenced yourself or publicly available data. In the second part of this tutorial we will describe how to perform a QC assessment for your NGS data.


In [ ]:
mkdir ngs
cd ngs

Now you can follow the instructions in the tutorial from here.

## Let’s get started!
Let's download `samtools`,`bcftools`and `htslib`.

Do this from commandline.

```
sudo yum install samtools bcftools htslib gnuplot
```

In [ ]:
samtools --help

In [ ]:
bcftools --help

This should return the help message for samtools and bcftools respectively.

## Download the required datasets

In [ ]:
wget https://github.com/cb2edu/CB2-101-BioComp/raw/2020/07-NGS_Intro_QC/data/example.tgz
tar -xvzf example.tgz

## FASTQ
FASTQ is a data format for raw unaligned sequencing reads. It is an extension to the FASTA file format, and includes a quality score for each base. Have a look at the example below, containing two reads:
  
```
@ERR007731.739 IL16_2979:6:1:9:1684/1
CTTGACGACTTGAAAAATGACGAAATCACTAAAAAACGTGAAAAATGAGAAATG
+
BBCBCBBBBBBBABBABBBBBBBABBBBBBBBBBBBBBABAAAABBBBB=@>B
@ERR007731.740 IL16_2979:6:1:9:1419/1
AAAAAAAAAGATGTCATCAGCACATCAGAAAAGAAGGCAACTTTAAAACTTTTC
+
BBABBBABABAABABABBABBBAAA>@B@BBAA@4AAA>.>BAA@779:AAA@A
```
   
We can see that for each read we get four lines:

   1. The read metadata, such as the read ID. Starts with `@` and, for paired-end Illumina reads, is terminated with /1 or /2 to show that the read is the member of a pair. 
   2. The read
   3. Starts with `+` and optionally contains the ID again
   4. The per base [Phred quality score](https://en.wikipedia.org/wiki/Phred_quality_score)

The quality scores range (in theory) from 1 to 94 and are encoded as [ASCII characters](https://en.wikipedia.org/wiki/ASCII). The first 32 ASCII codes are reserved for control characters which are not printable, and the 33rd is reserved for space. Neither of these can be used in the quality string, so we need to subtract 33 from whatever the value of the quality character is. For example, the ASCII code of “A” is 65, so the corresponding quality is:
   
```   
Q = 65 - 33 = 32
```
   
The Phred quality score `Q` relates to the base-calling error probability `P` as
   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; P = 10<sup>-Q/10</sup>   
   
The Phred quality score is a measure of the quality of base calls. For example, a base assigned with a Phred quality score of 30 tells us that there is a 1 in 1000 chance that this base was called incorrectly.

|Phred Quality Score|Probability of incorrect base call|Base call accuracy|
|---|---|---|
|10|1 in 10|90%|
|20|1 in 100|99%|
|30|1 in 1000|99.9%|
|40|1 in 10,000|99.99%|
|50|1 in 100,000|99.999%|
|60|1 in 1,000,000|99.9999%|

The following simple perl command will print the quality score value for an ASCII character. Try changing the "A" to another character, for example one from the quality strings above (e.g. @, = or B).

In [ ]:
perl -e 'printf "%d\n",ord("@")-33;'

### Exercises
__Q2.1: How many reads are there in the file example.fastq? (Hint: remember that `@` is a possible quality score. Is there something else in the header that is unique?)__

## SAM
[SAM (Sequence Alignment/Map)](https://samtools.github.io/hts-specs/SAMv1.pdf) format was developed by the 1000 Genomes Project group in 2009 and is a unified format for storing read alignments to a reference genome. SAM format is a standard format for storing NGS sequencing reads, base qualities, associated meta-data and alignments of the data to a reference genome. If no reference genome is available, the data can also be stored unaligned.

The files consist of a header section (optional) and an alignment section. The alignment section contains one record (a single DNA fragment alignment) per line describing the alignment between fragment and reference. Each record has 11 fixed columns and optional key:type:value tuples. Open the SAM/BAM file specification document [https://samtools.github.io/hts-specs/SAMv1.pdf](https://samtools.github.io/hts-specs/SAMv1.pdf) either in a web browser or you can find a copy in the QC directory as you may need to refer to it throughout this tutorial. 

Now let us have a closer look at the different parts of the SAM/BAM format. 

### Header Section

The header section of a SAM file looks like:

<insert header text>

```
@HD	VN:1.0	SO:coordinate
@SQ	SN:test_ref	LN:17637
@RG ID:ERR003612 PL:ILLUMINA LB:g1k-sc-NA20538-TOS-1 PI:2000 DS:SRP000540 SM:NA20538 CN:SC
```

Each line in the SAM header begins with an `@`, followed by a two-letter header record type code as defined in the [SAM/BAM format specification document](https://samtools.github.io/hts-specs/SAMv1.pdf). Each record type can contain meta-data captured as a series of key-value pairs in the format of ‘TAG:VALUE’.

#### Read groups
One useful record type is RG which can be used to describe each lane of sequencing. The RG code can be used to capture extra meta-data for the sequencing lane. Some common RG TAGs are:

* ID: SRR/ERR number
* PL: Sequencing platform
* PU: Run name
* LB: Library name
* PI: Insert fragment size
* SM: Individual/Sample
* CN: Sequencing centre

While most of these are self explanitory, insert fragment size may occasionally be negative. This simply indicates that the reads found are overlapping while its size is less than 2 x read length.

### Exercises

Look at the following line from the header of a SAM file and answering the questions that follow: 

```
@RG ID:ERR003612 PL:ILLUMINA LB:g1k-sc-NA20538-TOS-1 PI:2000 DS:SRP000540 SM:NA20538 CN:SC
```
   
__Q3: What does RG stand for?__

__Q4: What is the sequencing platform?__

__Q5: What is the sequencing centre?__

__Q6: What is the lane identifier?__

__Q7: What is the expected fragment insert size?__  

### Alignment Section
The alignment section of SAM files contains one line per read alignment, an example is

ERR005816.1408831	163	Chr1	19999970	23	40M5D30M2I28M	=	20000147	213 GGTGGGTGGATCACCTGAGATCGGGAGTTTGAGACTAGGTGG...   
<=@A@??@=@A@A>@BAA@ABA:>@<>=BBB9@@2B3<=@A@...   

Each of the lines are composed of multiple columns listed below. The first 11 columns are mandatory.

1. QNAME: Query NAME of the read or the read pair i.e. DNA sequence
2. FLAG: Bitwise FLAG (pairing, strand, mate strand, etc.)
3. RNAME: Reference sequence NAME
4. POS: 1-Based leftmost POSition of clipped alignment
5. MAPQ: MAPping Quality (Phred-scaled)
6. CIGAR: Extended CIGAR string (operations: MIDNSHPX=)
7. MRNM: Mate Reference NaMe (’=’ if same as RNAME)
8. MPOS: 1-Based leftmost Mate POSition
9. ISIZE: Inferred Insert SIZE
10. SEQ: Query SEQuence on the same strand as the reference
11. QUAL: Query QUALity (ASCII-33=Phred base quality)
12. OTHER: Optional fields


### Exercises
Let's have a look at example.sam. This file contains only a subset of the alignment section of a BAM-file that we will look closer at soon. Notice that we can use the standard UNIX operations like __cat__ on this file.

In [ ]:
cat example.sam

__Q8: What is the mapping quality of ERR003762.5016205? (Hint: can you use grep and awk to find this?)__

__Q9: What is the CIGAR string for ERR003814.6979522? (Hint: we will go through the meaning of CIGAR strings in the next section)__

__Q10: What is the inferred insert size of ERR003814.1408899?__

### CIGAR string
Column 6 of the alignment is the CIGAR string for that alignment. The CIGAR string provides a compact representation of sequence alignment. Have a look at the table below. It contains the meaning of all different symbols of a CIGAR string:

|Symbol |Meaning                                             |
|---    |---                                                 |
|M      |alignment match or mismatch                         |
|=      |sequence match                                      |
|X      |sequence mismatch                                   |
|I      |insertion into the read (sample sequenced)          |
|D      |deletion from the read (sample sequenced)           |
|S      |soft clipping (clipped sequences present in SEQ)    |
|H      |hard clipping (clipped sequences NOT present in SEQ)|
|N      |skipped region from the reference                   |
|P      |padding (silent deletion from padded reference)     |

Below are two examples describing the CIGAR string in more detail.
  
__Example 1:__  
Ref:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;ACGTACGTACGTACGT  
Read:&nbsp;&nbsp;ACGT-&nbsp;-&nbsp;-&nbsp;-&nbsp;ACGTACGA  
Cigar: 4M 4D 8M  

The first four bases in the read are the same as in the reference, so we can represent these as 4M in the CIGAR string. Next comes 4 deletions, represented by 4D, followed by 7 alignment matches and one alignment mismatch, represented by 8M. Note that the mismatch at position 16 is included in 8M. This is because it still aligns to the reference.

__Example 2:__  
Ref:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;ACTCAGTG-&nbsp;-&nbsp;GT  
Read:&nbsp;&nbsp;ACGCA-&nbsp;TGCAGTtagacgt  
Cigar: 5M 1D 2M 2I 2M 7S  

Here we start off with 5 alignment matches and mismatches, followed by one deletion. Then we have two more alignment matches, two insertions and two more matches. At the end, we have seven soft clippings, 7S. These are clipped sequences that are present in the SEQ (Query SEQuence on the same strand as the reference).

### Exercises
__Q11: What does the CIGAR from Q9 mean?__

__Q12: How would you represent the following alignment with a CIGAR string?__  
 
Ref:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;ACGT-&nbsp;-&nbsp;-&nbsp;-&nbsp;ACGTACGT  
Read:&nbsp;&nbsp;ACGTACGTACGTACGT  

### Flags
Column 2 of the alignment contains a combination of bitwise FLAGs describing the alignment. The following table contains the information you can get from the bitwise FLAGs:
   
|Hex  |Dec |Flag         |Description                                           |
|---  |--- |---          |---                                                   |
|0x1  |1   |PAIRED       |paired-end (or multiple-segment) sequencing technology|
|0x2  |2   |PROPER_PAIR  |each segment properly aligned according to the aligner|
|0x4  |4   |UNMAP        |segment unmapped                                      |
|0x8  |8   |MUNMAP       |next segment in the template unmapped                 |
|0x10 |16  |REVERSE      |SEQ is reverse complemented                           |
|0x20 |32  |MREVERSE     |SEQ of the next segment in the template is reversed   |
|0x40 |64  |READ1        |the first segment in the template                     |
|0x80 |128 |READ2        |the last segment in the template                      |
|0x100|256 |SECONDARY    |secondary alignment                                   |
|0x200|512 |QCFAIL       |not passing quality controls                          |
|0x400|1024|DUP          |PCR or optical duplicate                              |
|0x800|2048|SUPPLEMENTARY|supplementary alignment                               |

For example, if you have an alignment with FLAG set to 113, this can only be represented by decimal codes `64 + 32 + 16 + 1`, so we know that these four flags apply to the alignment and the alignment is paired-end, reverse complemented, sequence of the next template/mate of the read is reversed and the read aligned is the first segment in the template.

#### Primary, secondary and supplementary alignments
A read that aligns to a single reference sequence (including insertions, deletions, skips and clipping but not direction changes), is a __linear alignment__. If a read cannot be represented as a linear alignment, but instead is represented as a group of linear alignments without large overlaps, it is called a __chimeric alignment__. These can for instance be caused by structural variations. Usually, one of the linear alignments in a chimeric alignment is considered to be the __representative__ alignment, and the others are called __supplementary__.

Sometimes a read maps equally well to more than one spot. In these cases, one of the possible alignments is marked as the __primary__ alignment and the rest are marked as __secondary__ alignments.

## BAM
BAM (Binary Alignment/Map) format, is a compressed binary version of SAM. This means that, while SAM is human readable, BAM is only readable for computers. BAM was developed for fast processing and random access. To achieve this, BGZF (Block GZIP) compression is used for indexing. BAM files can be viewed using samtools, and will then have the same format as a SAM file.  The key features of BAM are:

* Can store alignments from most mappers
* Supports multiple sequencing technologies
* Supports indexing for quick retrieval/viewing
* Compact size (e.g. 112Gbp Illumina = 116GB disk space)
* Reads can be grouped into logical groups e.g. lanes, libraries, samples
* Widely supported by variant calling packages and viewers

Since BAM is a binary format, we can't use the standard UNIX operations directly on this file format. __Samtools__ is a set of programs for interacting with SAM and BAM files. Using the samtools view command, print the header of the BAM file:

In [ ]:
samtools view -H NA20538.bam

### Exercises
__Q13: What version of the human assembly was used to perform the alignments? (Hint: Can you spot this somewhere in the @SQ records?)__

__Q14: How many lanes are in this BAM file? (Hint: Do you recall what RG represents?)__

__Q15: What version of bwa was used to align the reads? (Hint: is there anything in the @PG record that looks like it could be a version tag?)__  

The output from running samtools view on a BAM file without any options is a headerless SAM file. This gets printed to STDOUT in the terminal, so we will want to pipe it to something. Let's have a look at the first read of the BAM file:

In [ ]:
samtools view NA20538.bam | head -n 1

__Q16: What is the name of the first read? (Hint: have a look at the alignment section if you can't recall the different fields)__

__Q17: What position does the alignment of the read start at?__

## Indexing
To allow for fast random access of regions in BAM and CRAM files, they can be indexed. The files must first be coordinate-sorted rather that sorted by read name. This can be done using __samtools sort__. If no options are supplied, it will by default sort by the left-most position of the reference.

In [ ]:
samtools sort -o NA20538_sorted.bam NA20538.bam

Now we can use __samtools index__ to create an index file (.bai) for our sorted BAM file:

In [ ]:
samtools index NA20538_sorted.bam

To look for reads mapped to a specific region, we can use __samtools view__ and specify the region we are interested in as: RNAME[:STARTPOS[-ENDPOS]]. For example, if we wanted to look at all the reads mapped to a region called chr4, we could use:

``samtools view alignment.bam chr4``

To look at the region on chr4 beginning at position 1,000,000 and ending at the end of the chromosome, we can do:

  ``samtools view alignment.bam chr4:1000000``
  
And to explore the 1001bp long region on chr4 beginning at position 1,000 and ending at position 2,000, we can use:

  ``samtools view alignment.bam chr4:1000-2000``

### Exercises 
__Q18: How many reads are mapped to region 20025000-20030000 on chromosome 1?__

In [ ]:
samtools view NA20538_sorted.bam 1:20025000-20030000 | wc -l

## VCF
The VCF file format was introduced to store variation data. VCF consists of tab-delimited text and is parsable by standard UNIX commands which makes it flexible and user-extensible. 

### VCF header
The VCF header consists of meta-information lines (starting with `##`) and a header line (starting with `#`). All meta-information lines are optional and can be put in any order, except for _fileformat_. This holds the information about which version of VCF is used and must come first. 

The meta-information lines consist of key=value pairs. Examples of meta-information lines that can be included are ##INFO, ##FORMAT and ##reference. The values can consist of multiple fields enclosed by `<>`. More information about these fields is available in the VCF specification [http://samtools.github.io/hts-specs/VCFv4.3.pdf](http://samtools.github.io/hts-specs/VCFv4.3.pdf). This can be accessed using a web browser and there is a copy in the QC directory.

#### Header line
The header line starts with `#` and consists of 8 required fields:

1. CHROM: an identifier from the reference genome
2. POS: the reference position
3. ID: a list of unique identifiers (where available)
4. REF: the reference base(s)
5. ALT: the alternate base(s)
6. QUAL: a phred-scaled quality score
7. FILTER: filter status
8. INFO: additional information

If the file contains genotype data, the required fields are also followed by a FORMAT column header, and then a number of sample IDs. The FORMAT field specifies the data types and order. Some examples of these data types are:

* GT: Genotype, encoded as allele values separated by either / or |
* DP: Read depth at this position for this sample
* GQ: Conditional genotype quality, encoded as a phred quality

### Body
In the body of the VCF, each row contains information about a position in the genome along with genotype information on samples for each position, all according to the fields in the header line.

## BCF

BCF is a compressed binary representation of VCF.

VCF can be compressed with BGZF (bgzip) and  indexed with TBI or CSI (tabix), but even compressed it can still be very big. For example, a compressed VCF with 3781 samples of human data will be 54 GB for chromosome 1, and 680 GB for the whole genome. VCFs can also be slow to parse, as text conversion is slow. The main bottleneck is the "FORMAT" fields. For this reason the BCF format was developed. 

In BCF files the fields are rearranged for fast access. The following images show the process of converting a VCF file into a BCF file. 

Bcftools comprises a set of programs for interacting with VCF and BCF files. It can be used to convert between VCF and BCF and to view or extract records from a region.

### bcftools view  
Let's have a look at the header of the file 1kg.bcf in the data directory. Note that bcftools uses __`-h`__ to print only the header, while samtools uses __`-H`__ for this. 

In [ ]:
bcftools view -h 1kg.bcf

Similarly to BAM, BCF supports random access, that is, fast retrieval from a given region. For this, the file must be indexed:

In [ ]:
bcftools index 1kg.bcf

Now we can extract all records from the region 20:24042765-24043073, using the __`-r`__ option. The __`-H`__ option will make sure we don't include the header in the output:

In [ ]:
bcftools view -H -r 20:24042765-24043073 1kg.bcf

### bcftools query  
The versatile __bcftools query__ command can be used to extract any VCF field. Combined with standard UNIX commands, this gives a powerful tool for quick querying of VCFs. Have a look at the usage options:

In [ ]:
bcftools query -h

Let's try out some useful options. As you can see from the usage, __`-l`__ will print a list of all the samples in the file. Give this a go:

In [ ]:
bcftools query -l 1kg.bcf

Another very useful option is __`-s`__ which allows you to extract all the data relating to a particular sample. This is a [common option](http://samtools.github.io/bcftools/bcftools.html#common_options) meaning it can be used for many bcftools commands, like `bcftools view`. Try this for sample HG00131:

In [ ]:
bcftools view -s HG00131 1kg.bcf | head -n 50

The format option, __`-f`__ can be used to select what gets printed from your query command. For example, the following will print the position, reference base and alternate base for sample HG00131, separated by tabs:

In [ ]:
bcftools query -f'%POS\t%REF\t%ALT\n' -s HG00131 1kg.bcf | head

Finally, let's look at the __`-i`__ option. With this option we can select only sites for which a particular expression is true. For instance, if we only want to look at sites that have more than 2 alternate alleles, we can use the following expression (piped to `head` to only show a subset of the output):

In [ ]:
bcftools query -f'%CHROM\t%POS\t%REF\t%ALT\n' -i 'AC[0]>2' 1kg.bcf | head

We use __`-i`__ with the expression `AC[0]>2`. AC is an info field that holds the __a__llele __c__ount. Some fields can hold multiple values, so we use `AC[0]>2` to indicate that we are looking for the first value (this is zero indexed, and hence starts at 0 instead of 1), and that this value should be > 2. To format our output, we use __`-f`__ to specify that we want to print the chromosome name and position.

There is more information about expressions on the bcftools manual page [http://samtools.github.io/bcftools/bcftools.html#expressions](http://samtools.github.io/bcftools/bcftools.html#expressions)

### Exercises
Now, try and answer the following questions about the file 1kg.bcf in the data directory. For more information about the different usage options you can open the bcftools query manual page [http://samtools.github.io/bcftools/bcftools.html#query](http://samtools.github.io/bcftools/bcftools.html#query) in a web browser.

__Q19: What version of the human assembly do the coordinates refer to?__ 

__Q20: How many samples are there in the BCF?__

__Q21: What is the genotype of the sample HG00107 at the position 20:24019472? (Hint: use the combination of -r, -s, and -f options)__

__Q22: How many positions are there with more than 10 alternate alleles? (Hint: use the -i filtering option)__

__Q23: In how many positions does HG00107 have a non-reference genotype and a read depth bigger than 10? (Hint: you can use pipes to combine bcftools queries)__   

In [ ]:
bcftools view -s HG00107 -H -i 'DP>10' 1kg.bcf | bcftools view -s HG00107 -H -i 'AC[0]>0' 1kg.bcf

# QC assessment of NGS data
QC is an important part of any analysis. In this section we are going to look at some of the metrics and graphs that can be used to assess the QC of NGS data. 


## Base quality
[Illumina sequencing](https://en.wikipedia.org/wiki/Illumina_dye_sequencing) technology relies on sequencing by synthesis. One of the most common problems with this is __dephasing__. For each sequencing cycle, there is a possibility that the replication machinery slips and either incorporates more than one nucleotide or perhaps misses to incorporate one at all. The more cycles that are run (i.e. the longer the read length gets), the greater the accumulation of these types of errors gets. This leads to a heterogeneous population in the cluster, and a decreased signal purity, which in turn reduces the precision of the base calling. The figure below shows an example of this.

Because of dephasing, it is possible to have high-quality data at the beginning of the read but really low-quality data towards the end of the read. In those cases you can decide to trim off the low-quality reads, for example using a tool called [Trimmomatic](http://www.usadellab.org/cms/?page=trimmomatic).




## Quality control of fastq files

Download the required dataset.

In [ ]:
wget https://github.com/cb2edu/CB2-101-BioComp/raw/2020/07-NGS_Intro_QC/data/rnaseq_data.tar.gz

Unzip the tarball

In [ ]:
tar -xvzf rnaseq_data.tar.gz

FASTQC is software that is used to calculate the quality of the fastq data.

In [ ]:
wget --no-check-certificate https://www.bioinformatics.babraham.ac.uk/projects/fastqc/fastqc_v0.11.9.zip
unzip fastqc_v0.11.9.zip

Check the help of fastqc.

In [ ]:
FastQC/fastqc -h

In [ ]:
mkdir -p rnaseq_quality
FastQC/fastqc -o rnaseq_quality -f fastq rnaseq_data/adrenal_R1.fq

  Open the analysis file.

In [ ]:
firefox rnaseq_quality/adrenal_R1_fastqc.html

A good example:
http://www.bioinformatics.babraham.ac.uk/projects/fastqc/bad_sequence_fastqc.html

A bad example:
http://www.bioinformatics.babraham.ac.uk/projects/fastqc/bad_sequence_fastqc.html

## Read trimming
We will use Trimmomatic for read trimming and adapter removal. Download Trimmomatic from: http://www.usadellab.org/cms/?page=trimmomatic
Generally, Illumina adapters are of two types: Nextera for WGS and exome sequencing and Truseq from RNAseq. We need to keep that in mind and provide the for trimming.

We used TruSeq3-PE adapters for clipping. Some other options are:
1. Adapters will have max 2 mismatches and will be clipped if a score of 30 is reached. 2. Remove leading and trailing N bases if quality is below 2.
3. Move with a 4 bp window and cutting where the average quality falls below 15.
4. After trmming remove all sequences whose length is below 30.

In [ ]:
wget http://www.usadellab.org/cms/uploads/supplementary/Trimmomatic/Trimmomatic-0.39.zip
unzip Trimmomatic-0.39.zip

In [ ]:
java -jar Trimmomatic-0.39/trimmomatic-0.39.jar PE \
rnaseq_data/adrenal_R1.fq rnaseq_data/adrenal_R2.fq \
adrenal_R1_trim.fq adrenal_R1_unmapped.fq \
adrenal_R2_trim.fq adrenal_R2_unmapped.fq \
ILLUMINACLIP:Trimmomatic-0.39/adapters/TruSeq3-PE.fa:2:30:10 \
LEADING:3 TRAILING:3 MINLEN:30

## Generating QC stats of bam files
Now let's try this out! We will generate QC stats for two lanes of Illumina paired-end sequencing data from yeast. The reads have already been aligned to the [Saccromyces cerevisiae reference genome](ftp://ftp.ensembl.org/pub/current_fasta/saccharomyces_cerevisiae/dna) to produce the BAM file lane1.sorted.bam.

Now we will use __`samtools stats`__ to generate the stats for the primary alignments. The option __`-f`__ can be used to filter reads with specific tags, while __`-F`__ can be used to _filter out_ reads with specific tags. The following command will include only primary alignments:

In [ ]:
samtools stats -F SECONDARY NA20538_sorted.bam \
    > NA20538.sorted.bam.bchk

Have a look at the first 47 lines of the statistics file that was generated:

In [ ]:
head -n 47 NA20538.sorted.bam.bchk

This file contains a number of useful stats that we can use to get a better picture of our data, and it can even be plotted with __`plot-bamstats`__, as you will see soon. First let's have a closer look at some of the different stats. Each part of the file starts with a `#` followed by a description of the section and how to extract it from the file. Let's have a look at all the sections in the file:

In [ ]:
grep ^'#' NA20538.sorted.bam.bchk | grep 'Use'

### Summary Numbers (SN)
This initial section contains a summary of the alignment and includes some general statistics. In particular, you can see how many bases mapped, and how much of the genome that was covered.

### Exercises

Now look at the output and try to answer the questions below.

__Q2: What is the total number of reads?__

__Q3: What proportion of the reads were mapped?__

__Q4: How many pairs were mapped to a different chromosome?__

__Q5: What is the insert size mean and standard deviation?__

__Q6: How many reads were paired properly?__

__Q7: How many reads have zero mapping quality?__

__Q8: Which reads (forward or reverse) have higher base quality on average?__  

### Generating QC plots
Finally, we will create some QC plots from the output of the stats command using the command __plot-bamstats__ which is included in the samtools package: 

In [ ]:
plot-bamstats -p NA20538-plots/ NA20538.sorted.bam.bchk

Now in your web browser open the file NA20538-plots/index.html to view the QC information. 

__Congratulations__ you have reached the end of the Data formats and QC tutorial! 